#Detecting Plagiarism using ngrams

### Importing main libraries

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.util import ngrams, pad_sequence, everygrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Initializing Training data file

In [ ]:
# importing the module
import wikipedia

In [ ]:
tp = input("Enter word string ")
tp

Enter word string Natural language processing


'Natural language processing'

In [ ]:
result = wikipedia.search(tp)
print(result)

['Natural language processing', 'Natural language', 'Natural-language understanding', 'Outline of natural language processing', 'Natural Language Toolkit', 'History of natural language processing', 'Natural-language user interface', 'BERT (language model)', 'Natural language generation', 'Natural-language programming']


In [ ]:
# wikipedia page object is created
page_object = wikipedia.page(tp)
page_object

<WikipediaPage 'Natural language processing'>

In [ ]:
ptrain = page_object.content
ptrain

'Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.  The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them. The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.\nChallenges in natural language processing frequently involve speech recognition, natural-language understanding, and natural-language generation.\n\n\n== History ==\n\nNatural language processing has its roots in the 1950s. Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, though at the time

In [ ]:
from google.colab import files

with open('examplenlp.txt', 'w') as f:
  f.write(ptrain)

files.download('examplenlp.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#uploading training data file
train_file = "/content/examplenlp.txt"
train_open = open(train_file, "r")
#read train data file
train_data = train_open.read()

### Exploring training data

In [ ]:
train_data

'Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.  The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them. The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.\nChallenges in natural language processing frequently involve speech recognition, natural-language understanding, and natural-language generation.\n\n\n== History ==\n\nNatural language processing has its roots in the 1950s. Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, though at the time

In [ ]:
#display length of training data
len(train_data)

33137

In [ ]:
#lexical richness
len(set(train_data))/len(train_data)

0.002534930742070797

In [ ]:
#corpus

In [ ]:
# example1 = "/content/cleaned_merged_fairy_tales_without_eos.txt"
# file1 = open(example1, "r")
# Read the file

# FileContent = file1.read()
# FileContent

### Preprocessing training data

In [ ]:
#convert to lowercase
train_lower = train_data.lower()

In [ ]:
#add preprocessing, remove punctuation and brackets
train_text = re.sub(r"\[.*\]|\{.*\}", "", train_lower)
train_text = re.sub(r'[^\w\s]', "", train_text)

In [ ]:
#stopwords initialize
nltk.download('stopwords')
stop_words_nltk = set(stopwords.words('english'))
print(stop_words_nltk)

{'just', 'hasn', 'him', 'here', "weren't", 're', 'most', 'ours', 'is', 'an', 'he', 'nor', 'if', 'wouldn', 'down', 'had', 'that', 'hadn', 'isn', 'the', 'but', 'our', "hadn't", 'couldn', 'them', 's', 'herself', 'does', 'there', 'was', "doesn't", 'wasn', 'what', "wouldn't", 'yours', 'over', 'weren', 'her', 'did', 'out', 'shouldn', "wasn't", 'while', 'this', 'needn', 'before', 'below', 'all', 'no', 'should', 'whom', 'm', 'hers', 'itself', 'a', 'in', 'your', 'd', 'll', 'under', 'and', 'through', 'up', 'into', 'both', 'which', 'himself', 'being', 'myself', "you'll", 'as', 'now', 'its', 'then', 'his', 'off', "haven't", 'be', 'it', "aren't", 'each', 'you', 'were', "shouldn't", 'after', 'about', 'do', 'between', 'who', "isn't", 'we', 'these', 'theirs', "you're", 'ain', 'mustn', 'why', "it's", "don't", 'against', 'mightn', 'yourselves', "you've", 'where', 'doesn', 'any', "should've", 'me', 'when', 'own', 't', 'o', 'themselves', 'very', 'further', "needn't", "won't", "you'd", "that'll", 'doing', 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#tokenize file
nltk.download('punkt')
tokenized_text = word_tokenize(train_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#remove stopwords from file
tokenized_text_without_stopwords = [i for i in tokenized_text if not i in stop_words_nltk]

In [ ]:
tokenized_text_without_stopwords

['natural',
 'language',
 'processing',
 'nlp',
 'subfield',
 'linguistics',
 'computer',
 'science',
 'artificial',
 'intelligence',
 'concerned',
 'interactions',
 'computers',
 'human',
 'language',
 'particular',
 'program',
 'computers',
 'process',
 'analyze',
 'large',
 'amounts',
 'natural',
 'language',
 'data',
 'goal',
 'computer',
 'capable',
 'understanding',
 'contents',
 'documents',
 'including',
 'contextual',
 'nuances',
 'language',
 'within',
 'technology',
 'accurately',
 'extract',
 'information',
 'insights',
 'contained',
 'documents',
 'well',
 'categorize',
 'organize',
 'documents',
 'challenges',
 'natural',
 'language',
 'processing',
 'frequently',
 'involve',
 'speech',
 'recognition',
 'naturallanguage',
 'understanding',
 'naturallanguage',
 'generation',
 'history',
 'natural',
 'language',
 'processing',
 'roots',
 '1950s',
 'already',
 '1950',
 'alan',
 'turing',
 'published',
 'article',
 'titled',
 'computing',
 'machinery',
 'intelligence',
 'prop

In [ ]:
#stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
stemmer= PorterStemmer()
print("Before Stemming:")
print(tokenized_text_without_stopwords)
print("After Stemming:")
l = ' '
for word in tokenized_text_without_stopwords:
  l = l + stemmer.stem(word) + ' '
print(l)

Before Stemming:
['natural', 'language', 'processing', 'nlp', 'subfield', 'linguistics', 'computer', 'science', 'artificial', 'intelligence', 'concerned', 'interactions', 'computers', 'human', 'language', 'particular', 'program', 'computers', 'process', 'analyze', 'large', 'amounts', 'natural', 'language', 'data', 'goal', 'computer', 'capable', 'understanding', 'contents', 'documents', 'including', 'contextual', 'nuances', 'language', 'within', 'technology', 'accurately', 'extract', 'information', 'insights', 'contained', 'documents', 'well', 'categorize', 'organize', 'documents', 'challenges', 'natural', 'language', 'processing', 'frequently', 'involve', 'speech', 'recognition', 'naturallanguage', 'understanding', 'naturallanguage', 'generation', 'history', 'natural', 'language', 'processing', 'roots', '1950s', 'already', '1950', 'alan', 'turing', 'published', 'article', 'titled', 'computing', 'machinery', 'intelligence', 'proposed', 'called', 'turing', 'test', 'criterion', 'intellige

In [ ]:
training_data = l

### Testing data uploading and preprocessing

In [ ]:
#testing data file
test_file = "/content/examplenlpcutphrased.txt"
test_open = open(test_file, "r")
#read test data file
test_data = test_open.read()

In [ ]:
test_data

'Natural language processing (NLP) is a branch of linguistics, computer science and artificial intelligence that deals with the interaction between computers and human language, specifically programming computers to process and analyze large amounts of natural language data. The goal is a computer that can "understand" the content of documents, including the contextual nuances of their language. The technology can then accurately extract information and insights from the documents and categorize and organize the documents themselves. \n Highly successful natural language processing systems developed in the 1960s were SHRDLU, a natural language system operating in limited "block worlds" with a limited vocabulary, and ELIZA, a Rogerian psychotherapist simulation  written by Joseph Weizenbaum between 196\n and 1966. Unaware of the thoughts or feelings of human, ELIZA sometimes offered surprisingly human interactions. If the "patient" crossed a very small database, ELIZA could give a gener

In [ ]:
#convert to lowercase
test_lower = test_data.lower()

In [ ]:
#add preprocessing, remove punctuation and brackets
test_text = re.sub(r"\[.*\]|\{.*\}", "", test_lower)
test_text = re.sub(r'[^\w\s]', "", test_text)

In [ ]:
#tokenize
tokenized_text_test = word_tokenize(test_text)

In [ ]:
#stopwords implementation
tokenized_text_without_stopwords_test = [i for i in tokenized_text_test if not i in stop_words_nltk]

In [ ]:
#stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
stemmer= PorterStemmer()
print("Before Stemming:")
print(tokenized_text_without_stopwords_test)
print("After Stemming:")
lt = ' '
for word in tokenized_text_without_stopwords_test:
  lt = lt + stemmer.stem(word) + ' '
print(lt)

Before Stemming:
['natural', 'language', 'processing', 'nlp', 'branch', 'linguistics', 'computer', 'science', 'artificial', 'intelligence', 'deals', 'interaction', 'computers', 'human', 'language', 'specifically', 'programming', 'computers', 'process', 'analyze', 'large', 'amounts', 'natural', 'language', 'data', 'goal', 'computer', 'understand', 'content', 'documents', 'including', 'contextual', 'nuances', 'language', 'technology', 'accurately', 'extract', 'information', 'insights', 'documents', 'categorize', 'organize', 'documents', 'highly', 'successful', 'natural', 'language', 'processing', 'systems', 'developed', '1960s', 'shrdlu', 'natural', 'language', 'system', 'operating', 'limited', 'block', 'worlds', 'limited', 'vocabulary', 'eliza', 'rogerian', 'psychotherapist', 'simulation', 'written', 'joseph', 'weizenbaum', '196', '1966', 'unaware', 'thoughts', 'feelings', 'human', 'eliza', 'sometimes', 'offered', 'surprisingly', 'human', 'interactions', 'patient', 'crossed', 'small', '

### NGRAM model

In [ ]:
#ngrams
n = 3

ngrams = list(everygrams(training_data, max_len=n))
print("Number of ngrams:", len(ngrams))

Number of ngrams: 58950


In [ ]:
from nltk.lm import WittenBellInterpolated

In [ ]:
#ngram language model
model = WittenBellInterpolated(n)
model.fit([ngrams], vocabulary_text=training_data)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 38 items>


In [ ]:
len(l)

19651

In [ ]:
len(lt)

4022

In [ ]:
# assign scores
scores = []
for i, item in enumerate(lt[n-1:]):
    s = model.score(item, lt[i:i+n-1])
    scores.append(s)

In [ ]:
scores_np = np.array(scores)

In [ ]:
scores_np

array([0.07124319, 0.06930945, 0.03414584, ..., 0.00183197, 0.00371482,
       0.14543789])

### Visualising results

In [ ]:
# set width and height
width = 8
height = np.ceil(len(lt)/width).astype("int32")
print("Width, Height:", width, ",", height)

Width, Height: 8 , 503


In [ ]:
# copy scores to rectangular blank array
a = np.zeros(width*height)
a[:len(scores_np)] = scores_np
diff = len(a) - len(scores_np)

In [ ]:
from scipy.ndimage import gaussian_filter

In [ ]:
# apply gaussian smoothing for aesthetics
a = gaussian_filter(a, sigma=1.0)

# reshape to fit rectangle
a = a.reshape(-1, width)

In [ ]:
# format labels
labels = [" ".join(lt[i:i+width]) for i in range(n-1, len(lt), width)]
labels_individual = [x.split() for x in labels]
labels_individual[-1] += [""]*diff
labels = [f"{x:60.60}" for x in labels]

In [ ]:
import plotly.graph_objects as go

In [ ]:
# create heatmap
fig = go.Figure(data=go.Heatmap(
                z=a, x0=0, dx=1,
                y=labels, zmin=0, zmax=1,
                customdata=labels_individual,
                hovertemplate='%{customdata} <br><b>Score:%{z:.3f}<extra></extra>',
                colorscale=""))
fig.update_layout({"height":height*28, "width":1000, "font":{"family":"Courier New"}})
fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()

In [ ]:
!jupyter nbconvert --to html "I068_nlp_ngram_plag_model_1.ipynb"

[NbConvertApp] Converting notebook I068_nlp_ngram_plag_model_1.ipynb to html
[NbConvertApp] Writing 644941 bytes to I068_nlp_ngram_plag_model_1.html
